<!--NOTEBOOK_HEADER-->
*This notebook contains course material from [FAT0413156](https://gmxavier.github.io/FAT0413156)
by Gilberto Xavier (gilberto.xavier at lps.ufrj.br); the content is available [on Github](https://github.com/gmxavier/FAT0413156.git).
The text is released under the [CC-BY-NC-SA-4.0 license](https://creativecommons.org/licenses/by-nc-sa/4.0/legalcode),
and code is released under the [MIT license](https://opensource.org/licenses/MIT).*

<!--NAVIGATION-->
< [Flow measurement](http://nbviewer.jupyter.org/github/gmxavier/FAT0413156/blob/master/notebooks/06.00-Flow.ipynb) | [Contents](toc.ipynb) | [Analytical measurement](http://nbviewer.jupyter.org/github/gmxavier/FAT0413156/blob/master/notebooks/08.00-Analytical.ipynb) ><p><a href="https://colab.research.google.com/github/gmxavier/FAT0413156/blob/master/notebooks/05.00-Temperature.ipynb" target="_parent"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://raw.githubusercontent.com/gmxavier/FAT0413156/master/notebooks/05.00-Temperature.ipynb" target="_parent"><img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>

# Control and safety valves

## Control valves

Reading: Tony R. Kuphaldt's [*Lessons in Industrial Instrumentation*: Chapter 27](https://www.ibiblio.org/kuphaldt/socratic/sinst/book/liii_2v32.pdf#page=2110)

Instructions: Click the link above and read this chapter. Chapter 27 describes the main control valve types used as final control elements.

Reading this chapter should take approximately ?? minutes.

Terms of use: This work is licensed under terms of the [CC-BY-4.0 license](http://creativecommons.org/licenses/by/4.0/). It is attributed to Tony R. Kuphaldt.

Supplementary material:
- [Control valve at Wikipedia](https://en.wikipedia.org/wiki/Control_valve)
- [Emerson's Control Valve Handbook](https://www.emerson.com/documents/automation/control-valve-handbook-en-3661206.pdf)

### Control valve sizing for incompressible fluids by hand

Reading: ISA's standard [*ISA-75.01.01-2007 (IEC 60534-2-1 Mod)*: Chapter 6](http://integrated.cc/cse/ISA_750101_SPBd.pdf#page=14).

Instructions: Click the link above and read the chapter. Chapter 6 provides athe relationships between flow rates, flow coefficients, related installation factors, and pertinent service conditions for control valves handling incompressible fluids.

Reading this section should take approximately ?? minutes.

Terms of use: This work is copyrighted by ISA (2007).

### Control valve sizing for incompressible fluids using Python

The function [`size_control_valve_l`](https://fluids.readthedocs.io/fluids.control_valve.html#rcbc94063f805-1) from the great Caleb Bell's [`fluids`](https://github.com/CalebBell/fluids) package calculates flow coefficient of a control valve passing a liquid according to ISA-75.01.01-2007.

From [*ISA-75.01.01-2007 (IEC 60534-2-1 Mod)* Example 1](http://integrated.cc/cse/ISA_750101_SPBd.pdf#page=51) for a globe, parabolic plug, flow-to-open valve passing water, we have:

In [1]:
from fluids import size_control_valve_l

size_control_valve_l(rho=965.4, Psat=70.1E3, Pc=22120E3, mu=3.1472E-4, P1=680E3, P2=220E3, Q=0.1, 
                     D1=0.15, D2=0.15, d=0.15, FL=0.9, Fd=0.46) # hand calculation gives Kv = 165 m3/h

164.9954763704956

From [*ISA-75.01.01-2007 (IEC 60534-2-1 Mod)* Example 2](http://integrated.cc/cse/ISA_750101_SPBd.pdf#page=52) for a ball, segmented ball, flow-to-open valve passing water, we have:

In [2]:
size_control_valve_l(rho=965.4, Psat=70.1E3, Pc=22120E3, mu=3.1472E-4, P1=680E3, P2=220E3, Q=0.1, 
                     D1=0.1, D2=0.1, d=0.1, FL=0.6, Fd=0.98) # hand calculation gives Kv = 238 m3/h

238.05817216710483

### Control valve sizing for compressible fluids by hand

Reading: ISA's standard [*ISA-75.01.01-2007 (IEC 60534-2-1 Mod)*: Chapter 7](http://integrated.cc/cse/ISA_750101_SPBd.pdf#page=16).

Instructions: Click the link above and read the chapter. Chapter 7 provides athe relationships between flow rates, flow coefficients, related installation factors, and pertinent service conditions for control valves handling compressible fluids.

Reading this section should take approximately ?? minutes.

Terms of use: This work is copyrighted by ISA (2007).

### Control valve sizing for compressible fluids using Python

The function [`size_control_valve_g`](https://fluids.readthedocs.io/fluids.control_valve.html#rcbc94063f805-1) from the great Caleb Bell's [`fluids`](https://github.com/CalebBell/fluids) package calculates flow coefficient of a control valve passing a liquid according to ISA-75.01.01-2007.

From [*ISA-75.01.01-2007 (IEC 60534-2-1 Mod)* Example 3](http://integrated.cc/cse/ISA_750101_SPBd.pdf#page=55) for non-choked gas flow with attached fittings and a rotary, eccentric plug, flow-to-open control valve passing carbon dioxide, we have:

In [3]:
from fluids import size_control_valve_g

size_control_valve_g(T=433., MW=44.01, mu=1.4665E-4, gamma=1.30, Z=0.988, P1=680E3, P2=310E3, Q=38/36., 
                     D1=0.08, D2=0.1, d=0.05,FL=0.85, Fd=0.42, xT=0.60) # hand calculation gives Kv = 72.1 m3/h

72.58664545391052

From [*ISA-75.01.01-2007 (IEC 60534-2-1 Mod)* Example 4](http://integrated.cc/cse/ISA_750101_SPBd.pdf#page=60) for a small flow trim sized tapered needle plug valve passing water, we have:

In [4]:
size_control_valve_g(T=320., MW=39.95, mu=5.625E-5, gamma=1.67, Z=1.0, P1=2.8E5, P2=1.3E5, Q=0.46/3600., 
                     D1=0.015, D2=0.015, d=0.015, FL=0.98, Fd=0.07, xT=0.8) # hand calculation gives Cv = 0.0184

0.016498765335995726

## Safety valves

Reading: Tony R. Kuphaldt's [*Lessons in Industrial Instrumentation*: Section 32.5](https://www.ibiblio.org/kuphaldt/socratic/sinst/book/liii_2v32.pdf#page=2687)

Instructions: Click the link above and read this section. Section 32.5 provides a glance about overpressure safety devices including safety valves.

Reading this section should take approximately ?? minutes.

Terms of use: This work is licensed under terms of the [CC-BY-4.0 license](http://creativecommons.org/licenses/by/4.0/). It is attributed to Tony R. Kuphaldt.

## Exercises

Try to solve the related problems listed [here](https://doc.lagout.org/electronics/Fundamentals%20of%20Industrial%20Instrumentation%20and%20Process%20Control%20%5Bby%20William%20Dunn%5D.pdf) on page 198.

<!--NAVIGATION-->
< [Flow measurement](http://nbviewer.jupyter.org/github/gmxavier/FAT0413156/blob/master/notebooks/06.00-Flow.ipynb) | [Contents](toc.ipynb) | [Analytical measurement](http://nbviewer.jupyter.org/github/gmxavier/FAT0413156/blob/master/notebooks/08.00-Analytical.ipynb) ><p><a href="https://colab.research.google.com/github/gmxavier/FAT0413156/blob/master/notebooks/05.00-Temperature.ipynb" target="_parent"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://raw.githubusercontent.com/gmxavier/FAT0413156/master/notebooks/05.00-Temperature.ipynb" target="_parent"><img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>